# Pratices Dashboard Map - Sales

In [1]:
#imports nescessarios
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("data/sales_data_sample.csv",encoding='latin-1')
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [3]:
usa_df = df[df['COUNTRY'] == 'USA']#Filtrar dados dos EUA

#Formatar datetime
usa_df['Data_Pedido'] = pd.to_datetime(usa_df['ORDERDATE'])
usa_df['Ano'] = usa_df['Data_Pedido'].dt.year
usa_df['Meses'] = usa_df['Data_Pedido'].dt.month_name()

In [4]:
usa_df.isnull().sum()

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2         901
CITY                   0
STATE                  0
POSTALCODE            76
COUNTRY                0
TERRITORY           1004
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
Data_Pedido            0
Ano                    0
Meses                  0
dtype: int64

__Removendo as variveis nulas__

In [5]:
drop_cols = ['ADDRESSLINE1', 'ADDRESSLINE2', 'POSTALCODE','CITY', 'TERRITORY', 'PHONE', 'CONTACTFIRSTNAME','CONTACTLASTNAME','CUSTOMERNAME','ORDERNUMBER']
usa_df = usa_df.drop(drop_cols, axis=1)

In [6]:
usa_df.isnull().sum()

QUANTITYORDERED    0
PRICEEACH          0
ORDERLINENUMBER    0
SALES              0
ORDERDATE          0
STATUS             0
QTR_ID             0
MONTH_ID           0
YEAR_ID            0
PRODUCTLINE        0
MSRP               0
PRODUCTCODE        0
STATE              0
COUNTRY            0
DEALSIZE           0
Data_Pedido        0
Ano                0
Meses              0
dtype: int64

In [7]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1(id='output_title', children=[], style={'text-align': 'center'}),

    dcc.Dropdown(id="ano_selecionado",
                options=[
                    {"label": "2003","value":2003},
                    {"label": "2004","value":2004},
                    {"label":"2005","value":2005}],
                multi=False,
                value=2003,
                style={'width':"40%"}
                ),

                html.Br(),
                html.Div(id='output_container',children=[]),
                html.Br(),
                
                dcc.Graph(id='mapa_de_vendas',figure={})
])

In [8]:
usa_df.columns

Index(['QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER', 'SALES', 'ORDERDATE',
       'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID', 'PRODUCTLINE', 'MSRP',
       'PRODUCTCODE', 'STATE', 'COUNTRY', 'DEALSIZE', 'Data_Pedido', 'Ano',
       'Meses'],
      dtype='object')

__Criando Callback__

In [9]:
@app.callback(
    [Output(component_id='output_container',component_property='children'),
    Output(component_id='output_title',component_property='children'),
    Output(component_id='mapa_de_vendas',component_property='figure')],
    [Input(component_id='ano_selecionado',component_property='value')]
)

def update_graph(ano_selecionado):
    container = (f"O ano selecionado foi: {ano_selecionado}")

    title = (f"Mapa de vendas nos EUA no ano  {ano_selecionado}")

    df_ =usa_df.copy()
    df_ =df_[df_["Ano"] == ano_selecionado]
    df_ =df_[df_["STATUS"]== "Shipped"]

    fig = px.choropleth(
        data_frame= df_,
        locationmode='USA-states',
        locations='STATE',
        scope="usa",
        color='SALES',
        hover_data=['STATE','SALES'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        template='plotly_dark'
    )
    return container, title, fig





In [10]:
if __name__ == '__main__':
    app.run_server(debug=True,port=2,use_reloader=False)